In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Build `states`
## Get margins of pf positions

In [ ]:
# Imports
import asyncio

import numpy as np
import pandas as pd
from ib_insync import MarketOrder

from utils import get_pickle, get_portfolio_with_margins, get_open_orders, make_equity_contracts, get_prices_with_ivs

In [ ]:
df_pfm = get_portfolio_with_margins(MARKET)

## Get underlying prices of portfolio option positions

In [ ]:
# Get underlying prices of portfolio options
conditions = df_pfm.secType == 'OPT'
opt_symbols = set(df_pfm[conditions].symbol) # unique set of symbols
und_contracts = make_equity_contracts(opt_symbols, MARKET)

In [ ]:
und_prices = asyncio.run(get_prices_with_ivs(port=PORT, input_contracts = und_contracts))

## Get open orders

In [ ]:
# Get open orders
with IB().connect(port=PORT) as ib:
    df_openords = asyncio.run(get_open_orders(ib)).set_index('conId')

# Unreaped
### A naked call or put option that doesn't have an open order to reap. [light-yellow]

In [ ]:
df_openords

# Unsowed

In [ ]:
# get unds, open orders and portfolio
unds = set(get_pickle(DATAPATH / 'unds.pkl').keys())
df_openorder, df_pf = asyncio.run(get_order_pf(PORT))

In [ ]:
# No orders to sow and no existing positions

options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'

long = df_pf.position > 0
long_options = long & options
df_pf[long_options]

unsowed = unds - set(df_openorder.symbol) - set(df_pf.symbol)
unsowed

# Orphaned

In [ ]:
# Long calls or puts without any underlying stock position
options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'
long = df_pf.position >= 1
long_stocks = long & stocks
long_options = long & options
orphaned = long_stocks & long_options

orphaned = df_pf[orphaned]
orphaned